In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O 

# import backend
import tensorflow as  tf
from keras import backend as K

# Model architecture
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Input,Dense, Dropout, Flatten, Conv2D
from keras.layers import MaxPool2D, Activation, MaxPooling2D
from keras.layers.normalization import BatchNormalization

# Data processing
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.utils import to_categorical
from keras.preprocessing import image

#Optimisation
from keras.optimizers import Adam

In [15]:
from keras.datasets import cifar10
(x_train_all, y_train_all), (x_test, y_test) = cifar10.load_data()

In [16]:
print("Number of training sample: ",x_train_all.shape[0])
print("Number of test samples: ", x_test.shape[0])

Number of training sample:  50000
Number of test samples:  10000


In [17]:
LABEL_NAMES = ['Plane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

VALIDATION_SIZE = 10000

In [18]:
LABEL_NAMES[y_train_all[10025][0]]

'Frog'

In [19]:
x_train_all = x_train_all /255.0
x_test = x_test /255.0 

In [20]:
# split training and validation set.
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, random_state=0, test_size=0.2)

In [21]:
x_val.shape

(10000, 32, 32, 3)

In [22]:
x_train.shape

(40000, 32, 32, 3)

In [23]:
# Convert to One Hot Encoding
from keras.utils import np_utils

y_train_ohe = np_utils.to_categorical(y_train, num_classes=10)
y_test_ohe = np_utils.to_categorical(y_test, num_classes=10)
y_val_ohe = np_utils.to_categorical(y_val, num_classes=10)

print(y_val_ohe)
y_val_ohe.shape

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]]


(10000, 10)

In [24]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
def create_cnn_model():
    image_input = Input(shape=(32, 32, 3))
    
    vgg_model  = VGG16(weights='imagenet',include_top=False, input_tensor=image_input)
    
    flatt = Flatten()(vgg_model.output)
    
    couche1 = Dense(128, activation='relu')(flatt) 
    couche1_normalization = BatchNormalization()(couche1)
    couche1_dropout = Dropout(0.2)(couche1_normalization)
    couche2 = Dense(64, activation='relu')(couche1_dropout)
    couche2_normalization = BatchNormalization()(couche2)
    output = Dense(10, activation='softmax', name='output')(couche2_normalization)     
    model = Model( image_input, output )
    return model

model = create_cnn_model()
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [25]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
                metrics=['accuracy'])

In [26]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Use Data Augmentation
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip= True)
es = EarlyStopping(patience=10, monitor='val_accuracy', mode='max')
mc = ModelCheckpoint('./weights.h5', monitor='val_accuracy', mode='max', save_best_only=True)

model.fit_generator(datagen.flow(x_train, y_train_ohe,batch_size = 32), steps_per_epoch = 1250, epochs=500, validation_data=[x_val, y_val_ohe], callbacks = [es,mc])
# Load The Best weights in the ModelCheckpoint
model.load_weights('./weights.h5')

# Predict The Test
preds = model.predict(x_val)
score_test = accuracy_score( y_val, np.argmax(preds, axis=1) )

print (' The test score : ', score_test)
print('')

Epoch 1/500
  36/1250 [..............................] - ETA: 18:35 - loss: 2.5271 - accuracy: 0.1571

KeyboardInterrupt: 

In [0]:
_, evaluate = model.evaluate(x_test, y_test_ohe, verbose=1)
print('>%.3f' % (evaluate * 100.0))

10000/10000 [==============================] - 3s 307us/step
>87.590


In [27]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 846763393942975699
]
